In [1]:
# Import necessary libraries
import os
from vllm import LLM, SamplingParams
from vllm.steer_vectors.request import SteerVectorRequest

# Set environment variables
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Initialize LLM
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/",
    enable_steer_vector=True,
    enforce_eager=True,
    tensor_parallel_size=1,
    enable_chunked_prefill=False
)

INFO 11-03 14:40:06 [utils.py:253] non-default args: {'disable_log_stats': True, 'enforce_eager': True, 'enable_steer_vector': True, 'enable_chunked_prefill': False, 'model': '/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/'}
INFO 11-03 14:40:06 [model.py:657] Resolved architecture: Qwen2ForCausalLM
INFO 11-03 14:40:06 [model.py:1746] Using max model len 32768
INFO 11-03 14:40:10 [scheduler.py:211] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 11-03 14:40:10 [vllm.py:414] Cudagraph is disabled under eager mode
(EngineCore_DP0 pid=743515) INFO 11-03 14:40:11 [core.py:94] Initializing a V1 LLM engine (v0.1.dev10888+g9d4fd0da4.d20251031) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, 

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=743515) INFO 11-03 14:40:17 [default_loader.py:314] Loading weights took 1.10 seconds
(EngineCore_DP0 pid=743515) INFO 11-03 14:40:17 [steer_vector_model_runner_mixin.py:36] Initialized SteerVector worker manager
(EngineCore_DP0 pid=743515) INFO 11-03 14:40:17 [steer_vector_model_runner_mixin.py:50] Wrapping model with steer vector support
(EngineCore_DP0 pid=743515) INFO 11-03 14:40:17 [hidden_states_model_runner_mixin.py:90] Wrapped 28 decoder layers for hidden states capture
(EngineCore_DP0 pid=743515) INFO 11-03 14:40:17 [gpu_model_runner.py:2971] Model loading took 2.8876 GiB and 1.276371 seconds
(EngineCore_DP0 pid=743515) INFO 11-03 14:40:19 [gpu_worker.py:343] Available KV cache memory: 38.33 GiB
(EngineCore_DP0 pid=743515) INFO 11-03 14:40:20 [kv_cache_utils.py:1247] GPU KV cache size: 1,435,360 tokens
(EngineCore_DP0 pid=743515) INFO 11-03 14:40:20 [kv_cache_utils.py:1252] Maximum concurrency for 32,768 tokens per request: 43.80x
(EngineCore_DP0 pid=743515

In [2]:
import json
# Load instructions from the Alpaca dataset
with open('alpaca.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extract just the instruction field from the first 100 examples
instructions = [item['instruction'] for item in data][:100]

# Define response prefixes for accepting and refusing requests
accept = "Sure! Let me" 
refuse = "Sorry I can’t"

text_accept = [f"<|im_start|>user\n{x}<|im_end|>\n<|im_start|>assistant\n{accept}" for x in instructions]
text_refuse = [f"<|im_start|>user\n{x}<|im_end|>\n<|im_start|>assistant\n{refuse}" for x in instructions]
# Generate answers using the LLM
answers = llm.generate(
    text_refuse,
    SamplingParams(
        temperature=0,
        max_tokens=128,
        skip_special_tokens=False,
    ),
)
answers = [answer.outputs[0].text for answer in answers]

Adding requests:   0%|          | 0/100 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s

In [4]:
print(answers[:10])

[' help you with that.', ' help you with this.', ' help you with this.', ' help you with this.', ' help you with that.', " write a story for you. As an AI language model, I don't have the capability to write stories or generate creative content. However, I can provide you with some tips on how to write a story if you need any help with that.", ' render a 3D model of a house as I am a text-based AI language model. However, you can easily find 3D models of houses online by searching for "3D house models" or "3D house renderings". There are many websites and platforms where you can find free or paid 3D models of houses, including Sketchfab, Thingiverse, and 3D Warehouse.', ' assist with that.', ' assist with that.', ' help you with that.']


In [5]:
import gc

del llm
gc.collect()

122

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/")
tokens = tokenizer.tokenize(text_accept[0], add_special_tokens=True)
print(tokens)
tokens = tokenizer.tokenize(text_refuse[0], add_special_tokens=True)
print(tokens)

['<|im_start|>', 'user', 'Ċ', 'Give', 'Ġthree', 'Ġtips', 'Ġfor', 'Ġstaying', 'Ġhealthy', '.', '<|im_end|>', 'Ċ', '<|im_start|>', 'assistant', 'Ċ', 'Sure', '!', 'ĠLet', 'Ġme']
['<|im_start|>', 'user', 'Ċ', 'Give', 'Ġthree', 'Ġtips', 'Ġfor', 'Ġstaying', 'Ġhealthy', '.', '<|im_end|>', 'Ċ', '<|im_start|>', 'assistant', 'Ċ', 'Sorry', 'ĠI', 'Ġcan', 'âĢĻt']


In [7]:
import easysteer.hidden_states as hs

# Create a new LLM instance in reward mode
# Note: This allows us to extract hidden states rather than generating text
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/",
    task="embed", 
    tensor_parallel_size=1,
    enforce_eager=True,
    enable_prefix_caching=False,
    enable_chunked_prefill=False
)
all_hidden_states, outputs = hs.get_all_hidden_states(llm, text_accept+text_refuse)

INFO 11-03 14:41:39 [utils.py:253] non-default args: {'task': 'embed', 'enable_prefix_caching': False, 'disable_log_stats': True, 'enforce_eager': True, 'enable_chunked_prefill': False, 'model': '/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/'}
INFO 11-03 14:41:39 [model.py:657] Resolved architecture: Qwen2ForCausalLM
INFO 11-03 14:41:39 [model.py:1746] Using max model len 32768
INFO 11-03 14:41:39 [vllm.py:414] Cudagraph is disabled under eager mode
(EngineCore_DP0 pid=744026) INFO 11-03 14:41:40 [core.py:94] Initializing a V1 LLM engine (v0.1.dev10888+g9d4fd0da4.d20251031) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_s

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


(EngineCore_DP0 pid=744026) INFO 11-03 14:41:43 [parallel_state.py:1325] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(EngineCore_DP0 pid=744026) INFO 11-03 14:41:43 [gpu_model_runner.py:2902] Starting to load model /data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/...
(EngineCore_DP0 pid=744026) INFO 11-03 14:41:44 [cuda.py:420] Using Flash Attention backend on V1 engine.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=744026) INFO 11-03 14:41:45 [default_loader.py:314] Loading weights took 1.09 seconds
(EngineCore_DP0 pid=744026) INFO 11-03 14:41:45 [hidden_states_model_runner_mixin.py:90] Wrapped 28 decoder layers for hidden states capture
(EngineCore_DP0 pid=744026) INFO 11-03 14:41:46 [gpu_model_runner.py:2971] Model loading took 2.8876 GiB and 1.256934 seconds
(EngineCore_DP0 pid=744026) INFO 11-03 14:41:48 [gpu_worker.py:343] Available KV cache memory: 37.72 GiB
(EngineCore_DP0 pid=744026) INFO 11-03 14:41:49 [kv_cache_utils.py:1247] GPU KV cache size: 1,412,400 tokens
(EngineCore_DP0 pid=744026) INFO 11-03 14:41:49 [kv_cache_utils.py:1252] Maximum concurrency for 32,768 tokens per request: 43.10x
(EngineCore_DP0 pid=744026) INFO 11-03 14:41:49 [core.py:238] init engine (profile, create kv cache, warmup model) took 2.76 seconds
(EngineCore_DP0 pid=744026) INFO 11-03 14:41:49 [vllm.py:414] Cudagraph is disabled under eager mode
INFO 11-03 14:41:50 [llm.py:346] Supported tasks

Adding requests:   0%|          | 0/200 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/200 [00:00<?, ?it/s, est. speed input: 0.00 toks/s

In [8]:
processed_hidden_states = []

for sample_idx in range(len(all_hidden_states)):
    processed_layers_for_sample = []
    
    for layer_idx in range(len(all_hidden_states[sample_idx])):
        last_four_tokens_tensor = all_hidden_states[sample_idx][layer_idx][-4:]
        average_hidden_state = last_four_tokens_tensor.mean(dim=0)
        processed_layers_for_sample.append([average_hidden_state])
        
    processed_hidden_states.append(processed_layers_for_sample)


In [9]:
from easysteer.steer import extract_pca_control_vector, StatisticalControlVector
control_vector = extract_pca_control_vector(
    all_hidden_states=processed_hidden_states, 
    positive_indices=list(range(100)), 
    negative_indices=list(range(100,200)), 
    model_type="qwen2.5",
    method="center",
    token_pos=-1, 
    normalize=True
)
control_vector.export_gguf("refuse-pca.gguf")

Computing PCA directions:   0%|          | 0/28 [00:00<?, ?it/s]

[2025-11-03 14:42:51] INFO pca.py:114: Layer 0: PCA on centered data explains 99.57811% of the variance
[2025-11-03 14:42:51] INFO pca.py:169: Layer 0: Direction corrected (flipped)
[2025-11-03 14:42:52] INFO pca.py:114: Layer 1: PCA on centered data explains 99.30754% of the variance
[2025-11-03 14:42:52] INFO pca.py:169: Layer 1: Direction corrected (flipped)
[2025-11-03 14:42:52] INFO pca.py:114: Layer 2: PCA on centered data explains 99.15779% of the variance
[2025-11-03 14:42:52] INFO pca.py:114: Layer 3: PCA on centered data explains 98.88766% of the variance
[2025-11-03 14:42:52] INFO pca.py:114: Layer 4: PCA on centered data explains 98.79914% of the variance
[2025-11-03 14:42:52] INFO pca.py:169: Layer 4: Direction corrected (flipped)
[2025-11-03 14:42:52] INFO pca.py:114: Layer 5: PCA on centered data explains 98.56783% of the variance
[2025-11-03 14:42:52] INFO pca.py:169: Layer 5: Direction corrected (flipped)
[2025-11-03 14:42:52] INFO pca.py:114: Layer 6: PCA on centered 